In [ ]:
import numpy
import pyfabm
import datetime
from matplotlib import pyplot

In [ ]:
import pickle
with open('../boxmodel/water_budget.pickle', 'rb') as f:
    water_budget_dt = pickle.load(f)
    water_budget = pickle.load(f)

In [ ]:
refdate = None
salt_path = '../boxmodel/salinity.dat'
salt_times = []
salts = []
elevs = []
with open(salt_path) as f:
    for l in f:
        date, salt, elev = l.rstrip('\n').split('\t')
        date = datetime.datetime.strptime(date, '%Y-%m-%d')
        if refdate is None:
            refdate = date
        salts.append(float(salt))
        elevs.append(float(elev))
        salt_times.append((date - refdate).days)
fig, ax = pyplot.subplots(figsize=(10,5))
dates = [refdate + datetime.timedelta(days=d) for d in salt_times]
ax.plot(dates, salts)
ax.grid(True)

In [ ]:
meteo_path = '../meteo.dat'
temp_times = []
temp_dates = []
temps = []
with open(meteo_path) as f:
    for l in f:
        items = l.rstrip('\n').split('\t')
        date = datetime.datetime.strptime(items.pop(0), '%Y-%m-%d %H:%M:%S')
        temp = float(items[-3])
        if refdate is None:
            refdate = date
        temps.append(temp)
        temp_dates.append(date)
        temp_times.append((date - refdate).days)
fig, ax = pyplot.subplots(figsize=(10,5))
ax.plot(temp_dates, temps)
ax.set_ylabel('temperature (degrees Celsius')
ax.grid(True)

In [ ]:
model = pyfabm.Model('fabm - Copy.yaml')

# Configure the environment
# Note: the set of environmental dependencies depends on the loaded biogeochemical model.
tempvar = model.dependencies['temperature']
tempvar.value = 27.
saltvar = model.dependencies['practical_salinity']
saltvar.value = 0.
model.cell_thickness = 10.

# Verify the model is ready to be used
assert model.start(), 'One or more model dependencies have not been fulfilled.'

outputnames = 'deb/N_b', 'deb/N_j', 'deb/N_p', 'deb/N_a', 'deb/waste', 'practical_salinity', 'temperature', 'total_energy_at_interfaces_calculator/result', 'deb/Et_b', 'deb/Et_j', 'deb/Et_p', 'deb/Et_a', 'deb/cysts', 'deb/f'

# Time derivative
dt = 360.
nday = salt_times[-1] #365
nspinup = 365*3
n = int(nday * 86400 / dt)
output2dat = {}
for name in outputnames:
    output2dat[name] = (model.variables[name], numpy.empty((n,)))
t = numpy.arange(n) * dt / 86400.
model_temps = numpy.interp(t, temp_times, temps)
model_salts = numpy.interp(t, salt_times, salts)
for _ in range(3):
    for i in range(int(nspinup * 86400 / dt)):
        saltvar.value = model_salts[i]
        tempvar.value = model_temps[i]
        model.state += model.getRates() * dt
for i in range(n):
    saltvar.value = model_salts[i]
    tempvar.value = model_temps[i]
    model.state += model.getRates() * dt
    for outputvar, dat in output2dat.values():
        dat[i] = outputvar.value


In [ ]:
# Plot results
E_G = model.parameters['deb/E_G'].value
outputnames = 'total_energy_at_interfaces_calculator/result', #'deb/N_e', 'deb/N_j', 'deb/N_a' #'total_energy_at_interfaces_calculator/result', #'deb/N_e', 'deb/N_j', 'deb/N_a', #'deb/Et_a', 'deb/Et_e', 'deb/Et_j', #'deb/waste', 'deb/N_a',
outputnames = 'deb/N_b', 'deb/N_j', 'deb/N_p', 'deb/N_a', 'deb/cysts'
#outputnames = 'deb/Et_b', 'deb/Et_j', 'deb/Et_p', 'deb/Et_a' #, 'deb/cysts'
#outputnames = 'deb/f',
outputsum = sum([output2dat[name][1] for name in outputnames])
nsave = 1
dates = [refdate + datetime.timedelta(days=d) for d in t]
fig, ax = pyplot.subplots(figsize=(10,5))
for name in outputnames:
    outputvar, dat = output2dat[name]
    ax.plot(dates[::nsave], dat[::nsave], label=outputvar.long_name)
#ax.plot(dates[::nsave], outputsum[::nsave], label='sum')
ax.grid()
ax.set_xlim(dates[0], dates[-1])
#ax.set_ylim(None, 1000)
#ax.set_xlim(datetime.datetime(1999,1,1), datetime.datetime(2003,1,1))
ax.legend()


In [ ]:
model_dates = [refdate + datetime.timedelta(days=d) for d in t]

crit_dt1, crit_dt2 = datetime.datetime(2000, 1, 1), datetime.datetime(2002, 1, 1)

fig, (ax1, ax2, ax3) = pyplot.subplots(figsize=(8,8), nrows=3, sharex=True)
dates = [refdate + datetime.timedelta(days=d) for d in salt_times]
ax1.plot(water_budget_dt, water_budget[:, 3], color='C1')
ax1.plot(dates, elevs, color='C0')
ax1.grid(True)
#ax1.set_ylabel('water level (m)')
ax1.set_xlim(dates[0], dates[-1])
ax1.set_ylabel('water level (m)')
ax1.axvspan(crit_dt1, crit_dt2, color='r', alpha=0.2)

ax2.plot(dates, salts)
ax2.grid(True)
#ax2.set_ylabel('salinity (g/L)')
ax2.set_ylabel('salinity (g/L)')
ax2.set_xlim(dates[0], dates[-1])
ax2.axvspan(crit_dt1, crit_dt2, color='r', alpha=0.2)

dat = output2dat['deb/Et_b'][1] + output2dat['deb/Et_j'][1] + output2dat['deb/Et_p'][1] + output2dat['deb/Et_a'][1]
ax3.plot(model_dates[::nsave], dat / E_G, label='sum')
#dat = output2dat['deb/cysts'][1]
#ax3.plot(model_dates[::nsave], dat[::nsave], label='cysts')
#dat = output2dat['deb/N_j'][1] + output2dat['deb/N_p'][1]
#ax3.plot(model_dates[::nsave], dat[::nsave], label='nauplii + juveniles')
#dat = output2dat['deb/N_a'][1]
#ax3.plot(model_dates[::nsave], dat[::nsave], label='adults')
#ax3.legend()
ax3.grid(True)
ax3.set_xlabel('year')
ax3.set_ylabel('Artemia (rel. biomass)')
#ax3.set_ylabel('Artemia (g/m3)')
ax3.set_xlim(dates[0], dates[-1])
ax3.axvspan(crit_dt1, crit_dt2, color='r', alpha=0.2)
fig.savefig('box_deb.png', dpi=300)
